In [30]:
%pip install llama_index

In [13]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [15]:
import sqlite3
import os
import pandas as pd
from llama_index import SimpleDirectoryReader, LLMPredictor, PromptHelper, ServiceContext, GPTVectorStoreIndex, OpenAIEmbedding, StorageContext, load_index_from_storage
from langchain import OpenAI

In [16]:
#data reading
data = pd.read_csv("knowledge\mobile_prices.csv")

#Converting into CSV --> Database
conn = sqlite3.connect('database.db')
data.to_sql('table_name', conn, if_exists='replace')

1836

In [17]:
def retrieve_data(database_file):
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()
    query = "SELECT * FROM table_name"
    data = cursor.fetchall()
    conn.close()
    return data

In [18]:
def sql_query(q):
    cursor = conn.cursor()
    cursor.execute(q)
    conn.commit()

    if q.strip().upper().startswith('SELECT'):
        # For SELECT queries, return the results
        results = cursor.fetchall()
        return results
    elif q.strip().upper().startswith('INSERT'):
        # For INSERT queries, return a success message
        return "Data inserted successfully!"
    else:
        # For other queries (UPDATE, DELETE, etc.), return None
        return "ELSE Block executed"


In [19]:
import openai
os.environ["OPENAI_API_KEY"] = "****************************************" 
openai.api_key = os.environ["OPENAI_API_KEY"]


In [20]:
def create_index(database_file):
    max_input = 4096
    tokens = 246
    chunk_size = 600
    max_chunk_overlap = 0.1
    
    # Initialize PromptHelper
    promptHelper = PromptHelper(max_input, tokens, max_chunk_overlap, chunk_size_limit=chunk_size)
    
    # Initialize LLMPredictor with the specified LLM model
    llmPredictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-ada-001", max_tokens=tokens, openai_api_key="************************************************"))
    
    
    # Retrieve data from the database
    data = retrieve_data(database_file)
    
    # Initialize ServiceContext with LLMPredictor and PromptHelper
    service_context = ServiceContext.from_defaults(llm_predictor=llmPredictor, prompt_helper=promptHelper)
    
    # Create vector index
    embed_model = OpenAIEmbedding(embed_batch_size=4)
    vectorIndex = GPTVectorStoreIndex.from_documents(documents=data, service_context=service_context, embed_model=embed_model)
    
    # Persist the vector index to the specified directory
    vectorIndex.storage_context.persist(persist_dir="./Store")
    
    return vectorIndex

In [21]:
def answer_question(question):
    # Initialize StorageContext with the specified persist directory
    storage_context = StorageContext.from_defaults(persist_dir="./Store")
    
    # Load the index from the storage directory
    index = load_index_from_storage(storage_context)
    
    # Create a query engine from the loaded index
    query_engine = index.as_query_engine()
    
    # Query the question using the query engine
    response = query_engine.query(question)
    
    return response

In [22]:
database_file = "C:/Users/venka/OneDrive/Desktop/Work/CSVdql bot_2/database.db"
vector_index = create_index(database_file)


In [23]:
question = "which is the best phone?" #<------------------- ERROR ------- I am not getting any proper anwer as output 
response = answer_question(question)   
print(response)

None


In [29]:
##INPUT:
i=1
while(i):
    print("""
            0 - STOP
            1 - Normal Query
            2 - SQL Query""")
    i = int(input())
    if(i==0):
        break
    elif(i==1):
        tex = input("Enter the Question: ")
        print(answer_question(tex))
    elif(i==2):
        tex = input("Enter the Question: ")
        print(sql_query(tex))
    else:
        print("invalid input, TRY AGAIN!")


            0 - STOP
            1 - Normal Query
            2 - SQL Query
[(0, 'POCO C50 (Royal Blue, 32 GB)', 4.2, '33,561', '2 GB RAM', '32 GB ROM', '8MP Dual Camera', '5MP Front Camera', '5000 mAh', 'Mediatek Helio A22 Processor, Upto 2.0 GHz Processor', '₹5,649', '2023-06-17'), (1, 'POCO M4 5G (Cool Blue, 64 GB)', 4.2, '77,128', '4 GB RAM', '64 GB ROM', '50MP + 2MP', '8MP Front Camera', '5000 mAh', 'Mediatek Dimensity 700 Processor', '₹11,999', '2023-06-17'), (2, 'POCO C51 (Royal Blue, 64 GB)', 4.3, '15,175', '4 GB RAM', '64 GB ROM', '8MP Dual Rear Camera', '5MP Front Camera', '5000 mAh', 'Helio G36 Processor', '₹6,999', '2023-06-17'), (3, 'POCO C55 (Cool Blue, 64 GB)', 4.2, '22,621', '4 GB RAM', '64 GB ROM', '50MP Dual Rear Camera', '5MP Front Camera', '5000 mAh', 'Mediatek Helio G85 Processor', '₹7,749', '2023-06-17'), (4, 'POCO C51 (Power Black, 64 GB)', 4.3, '15,175', '4 GB RAM', '64 GB ROM', '8MP Dual Rear Camera', '5MP Front Camera', '5000 mAh', 'Helio G36 Processor', '₹6,